In [201]:
import pygsti
import numpy as np
from scipy.linalg import expm
from pygsti.circuits import Circuit
from matplotlib import pyplot as plt

In [202]:
# Gell-Mann matrices
gellmann_matrices = [
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
    np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]),
    np.array([[0, -1j, 0], [1j, 0, 0], [0, 0, 0]]),
    np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]]),
    np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]]),
    np.array([[0, 0, -1j], [0, 0, 0], [1j, 0, 0]]),
    np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]),
    np.array([[0, 0, 0], [0, 0, -1j], [0, 1j, 0]]),
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, -2]])
]

gellmann_8_12 = np.array([[-2, 0, 0], [0, 1, 0], [0, 0, 1]])

In [203]:
# unitary models 
# we ignore axis error 
def modelX01(theta, gamma):
    return expm(-(1j/2)*((np.pi/2 + theta)*gellmann_matrices[1] + gamma*gellmann_matrices[8]))

def modelZ01():
    return expm(-(1j*np.pi/4)*gellmann_matrices[3])

Z12_gen = np.array([[0, 0, 0], [0, 1, 0], [0, 0, -1]])

def modelZ12():
    return expm(-(1j*np.pi/4)*Z12_gen)

def modelX12(theta, gamma):
    return expm(-(1j/2)*((np.pi/2 + theta)*gellmann_matrices[4]  + gamma*gellmann_8_12))

def modelCZ(phis):
    return np.diag([1]+[np.exp(-1j*phi) for phi in phis])



In [207]:
def parse_error_vector(x):
    info = {
        'single_qutrit': {
            'Q1': {
                'X01' : x[0],
                'phase01': x[1],
                'X12' : x[2], 
                'phase12': x[3],
            },
            'Q2': {
                'X01' : x[4],
                'phase01': x[5],
                'X12' : x[6],
                'phase12': x[7],
            }
        },
        'two_qutrit': {
            'phi1': x[8],
            'phi2': x[9],
            'phi3': x[10],
            'phi4': x[11],
            'phi5': x[12],
            'phi6': x[13],
            'phi7': x[14],
            'phi8': x[15]
        }  
    }
    return info

def random_error_vector(single_qutrit_rates, two_qutrit_rates):
    q1_vec = np.random.multivariate_normal(np.zeros(4), np.eye(4)*single_qutrit_rates)
    q2_vec = np.random.multivariate_normal(np.zeros(4), np.eye(4)*single_qutrit_rates)
    two_qubit_vec = np.random.multivariate_normal(np.zeros(8), np.eye(8)*two_qutrit_rates)
    return np.concatenate((q1_vec, q2_vec, two_qubit_vec))


In [208]:
from pygsti.tools import unitary_to_std_process_mx
from pygsti.modelmembers.operations import EmbeddedOp, FullArbitraryOp

In [210]:
from pygsti.baseobjs import ExplicitStateSpace
from pygsti.models import ExplicitOpModel
from pygsti.models.modelconstruction import create_spam_vector
from pygsti.modelmembers.povms import UnconstrainedPOVM, FullPOVMEffect
from pygsti.modelmembers.states import FullState
from pygsti.tools import change_basis
from pygsti.baseobjs import Basis



def label_to_spam_vec(lbl):
    if lbl == '00':
        rho = create_spam_vector('0', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '01':
        rho = create_spam_vector('1', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '02':
        rho = create_spam_vector('2', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '10':
        rho = create_spam_vector('3', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '11':
        rho = create_spam_vector('4', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '12':
        rho = create_spam_vector('5', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '20':
        rho = create_spam_vector('6', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '21':
        rho = create_spam_vector('7', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    elif lbl == '22':
        rho = create_spam_vector('8', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81))
    return FullPOVMEffect(rho, basis=Basis.cast('gm', 81))

def make_model(error_vector, single_qutrit_depol, two_qutrit_depol):
    joint_state_space = ExplicitStateSpace([('Q1','Q2')], [(3,3)])
    ss_q1 = ExplicitStateSpace('Q1', 3)
    ss_q2 = ExplicitStateSpace('Q2', 3)

    model = ExplicitOpModel(joint_state_space)
    gm  = Basis.cast('gm',  81)
    gm9 = Basis.cast('gm', 9)

    # Add state prep and measurement
    
    model['rho0'] =  FullState(create_spam_vector('0', ExplicitStateSpace([('Q1','Q2')], [(3,3)]), basis=Basis.cast('gm', 81)), basis=gm)
    model['Mdefault'] = UnconstrainedPOVM({ '00' : label_to_spam_vec('00'),
                                            '01' : label_to_spam_vec('01'),
                                            '02' : label_to_spam_vec('02'),
                                            '10' : label_to_spam_vec('10'),
                                            '11' : label_to_spam_vec('11'),
                                            '12' : label_to_spam_vec('12'),
                                            '20' : label_to_spam_vec('20'),
                                            '21' : label_to_spam_vec('21'),
                                            '22' : label_to_spam_vec('22')}, evotype='densitymx')

    # Parse error vector
    errors = parse_error_vector(error_vector)
    x01_Q1 = errors['single_qutrit']['Q1']['X01']
    x12_Q1 = errors['single_qutrit']['Q1']['X12']
    x01_Q2 = errors['single_qutrit']['Q2']['X01']
    x12_Q2 = errors['single_qutrit']['Q2']['X12']
    
    phase01_Q1 = errors['single_qutrit']['Q1']['phase01']
    phase12_Q1 = errors['single_qutrit']['Q1']['phase12']
    phase01_Q2 = errors['single_qutrit']['Q2']['phase01']
    phase12_Q2 = errors['single_qutrit']['Q2']['phase12']

    phi1 = errors['two_qutrit']['phi1']
    phi2 = errors['two_qutrit']['phi2']
    phi3 = errors['two_qutrit']['phi3']
    phi4 = errors['two_qutrit']['phi4']
    phi5 = errors['two_qutrit']['phi5']
    phi6 = errors['two_qutrit']['phi6']
    phi7 = errors['two_qutrit']['phi7']
    phi8 = errors['two_qutrit']['phi8']
    phis = [phi1, phi2, phi3, phi4, phi5, phi6, phi7, phi8]


    # Define single qutrit gates
    X01_Q1_unitary = unitary_to_std_process_mx(modelX01(x01_Q1, phase01_Q1))
    Z01_Q1_unitary = unitary_to_std_process_mx(modelZ01())
    X12_Q1_unitary = unitary_to_std_process_mx(modelX12(x12_Q1, phase12_Q1))
    Z12_Q1_unitary = unitary_to_std_process_mx(modelZ12())

    X01_Q1_mat = change_basis(X01_Q1_unitary, 'std', gm9)
    Z01_Q1_mat = change_basis(Z01_Q1_unitary, 'std', gm9)
    X12_Q1_mat = change_basis(X12_Q1_unitary, 'std', gm9)
    Z12_Q1_mat = change_basis(Z12_Q1_unitary, 'std', gm9)

    X01_Q1_gate = FullArbitraryOp(X01_Q1_mat, evotype='densitymx', basis=gm9)
    Z01_Q1_gate = FullArbitraryOp(Z01_Q1_mat, evotype='densitymx', basis=gm9)
    X12_Q1_gate = FullArbitraryOp(X12_Q1_mat, evotype='densitymx', basis=gm9)
    Z12_Q1_gate = FullArbitraryOp(Z12_Q1_mat, evotype='densitymx', basis=gm9)

    X01_Q1_gate.depolarize(single_qutrit_depol)
    Z01_Q1_gate.depolarize(single_qutrit_depol)
    X12_Q1_gate.depolarize(single_qutrit_depol)
    Z12_Q1_gate.depolarize(single_qutrit_depol)

    X01_Q2_unitary = unitary_to_std_process_mx(modelX01(x01_Q2, phase01_Q2))
    Z01_Q2_unitary = unitary_to_std_process_mx(modelZ01())
    X12_Q2_unitary = unitary_to_std_process_mx(modelX12(x12_Q2, phase12_Q2))
    Z12_Q2_unitary = unitary_to_std_process_mx(modelZ12())

    X01_Q2_mat = change_basis(X01_Q2_unitary, 'std', gm9)
    Z01_Q2_mat = change_basis(Z01_Q2_unitary, 'std', gm9)
    X12_Q2_mat = change_basis(X12_Q2_unitary, 'std', gm9)
    Z12_Q2_mat = change_basis(Z12_Q2_unitary, 'std', gm9)

    X01_Q2_gate = FullArbitraryOp(X01_Q2_mat, evotype='densitymx', basis=gm9)
    Z01_Q2_gate = FullArbitraryOp(Z01_Q2_mat, evotype='densitymx', basis=gm9)
    X12_Q2_gate = FullArbitraryOp(X12_Q2_mat, evotype='densitymx', basis=gm9)
    Z12_Q2_gate = FullArbitraryOp(Z12_Q2_mat, evotype='densitymx', basis=gm9)
                               
    X01_Q2_gate.depolarize(single_qutrit_depol)
    Z01_Q2_gate.depolarize(single_qutrit_depol)
    X12_Q2_gate.depolarize(single_qutrit_depol)
    Z12_Q2_gate.depolarize(single_qutrit_depol)

    # Define two qutrit gates
    CZ_unitary = unitary_to_std_process_mx(modelCZ(phis))
    CZ_mat = change_basis(CZ_unitary, 'std', gm)
    CZ = FullArbitraryOp(CZ_mat, evotype='densitymx', basis=gm)
    CZ.depolarize(two_qutrit_depol)

    # Add gates to model
    model.operations[('G_X01', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), X01_Q1_gate)
    model.operations[('G_Z01', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), Z01_Q1_gate)
    model.operations[('G_X12', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), X12_Q1_gate)
    model.operations[('G_Z12', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), Z12_Q1_gate)
    model.operations[('G_X01', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), X01_Q2_gate)
    model.operations[('G_Z01', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), Z01_Q2_gate)
    model.operations[('G_X12', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), X12_Q2_gate)
    model.operations[('G_Z12', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), Z12_Q2_gate)

    model.operations['G_CZ'] = CZ

    return model

In [211]:
prep_dict = {
    '00' : ([]), 
}

meas_dict = {
    '00' : ([]),
}


gate_dict = {
    'G_CZ' : ([('Gcz', 'Q1', 'Q2')]),
    'G_X01' : ([('G_X01(Q1)', 'Q1')]),
    'G_Z01' : ([('G_Z01(Q1)', 'Q1')]),
    'G_X12' : ([('G_X12(Q1)', 'Q1')]),
    'G_Z12' : ([('G_Z12(Q1)', 'Q1')]),
    'G_X01' : ([('G_X01(Q2)', 'Q2')]),
    'G_Z01' : ([('G_Z01(Q2)', 'Q2')]),
    'G_X12' : ([('G_X12(Q2)', 'Q2')]),
    'G_Z12' : ([('G_Z12(Q2)', 'Q2')]),
}

def make_circuit(gate, prep_label, meas_label, depth):
    prep_circ = prep_dict[prep_label]
    meas_circ = meas_dict[meas_label]
    return Circuit(['rho0'] + prep_circ + [gate]*depth + meas_circ + ['Mdefault'], line_labels=['Q1', 'Q2'])

In [212]:
x = random_error_vector(0, 0)
model = make_model(x, 0.00, 0.0)

In [216]:
circ = make_circuit(('G_X01', 'Q2'), '00', '00', 1)
print(circ)

Qubit Q1 ---|rho0|-|     |-|Mdefault|---
Qubit Q2 ---|rho0|-|G_X01|-|Mdefault|---



In [217]:
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.8421827970787253),
                  (('01',), -0.15781720292127477),
                  (('02',), 0.1222600274416637),
                  (('10',), 0.18224414530417493),
                  (('11',), 0.3966210178926214),
                  (('12',), -0.0879299970281064),
                  (('20',), -0.2133957811342858),
                  (('21',), -0.08416500663351895),
                  (('22',), -8.326672684688674e-17)])